In [2]:
using DifferentialEquations, Plots, LaTeXStrings, Printf, ProgressMeter, FFTW

In [3]:
# Define the simple pendulum ODE
function pendulum!(du, u, p, t)
    θ, ω = u
    du[1] = ω               # dθ/dt = ω
    du[2] = -sin(θ)         # dω/dt = -sin(θ)
end

pendulum! (generic function with 1 method)

In [4]:
# Parameters
n_pendula = 5000           # Number of pendula
tspan = (0.0, 100.0)        # Time span
fps = 30                   # Frames per second for animation
total_frames = 300;         # Total frames in animation

In [5]:
# Create initial conditions that thoroughly sample phase space
θ0 = range(-π, π, length=ceil(Int, sqrt(n_pendula)))
ω0 = range(-2.0, 2.0, length=ceil(Int, sqrt(n_pendula)))

# Create all possible combinations and flatten
all_conditions = vec([[θ, ω] for θ in θ0, ω in ω0])

# Alternate between beginning and end to mix positive/negative ω points
n_total = length(all_conditions)
n_use = min(n_total, n_pendula)
indices = Int[]
i, j = 1, n_total

while length(indices) < n_use
    if i <= j
        push!(indices, i)
        i += 1
    end
    if length(indices) < n_use && j >= i
        push!(indices, j)
        j -= 1
    end
end

initial_conditions = all_conditions[indices][1:n_pendula];

In [6]:
# Set up the ODE problems
problems = [ODEProblem(pendulum!, ic, tspan) for ic in initial_conditions]

# Solve all ODEs
solutions = [solve(prob, Tsit5(), saveat=range(tspan[1], tspan[2], length=total_frames))
             for prob in problems];

In [7]:
# Helper function for meshgrid (similar to MATLAB's meshgrid)
function meshgrid(x, y)
    X = [i for i in x, j in y]
    Y = [j for i in x, j in y]
    return X, Y
end

meshgrid (generic function with 1 method)

In [8]:
#=
# Prepare the animation
println("Creating animation...")
Plots.theme(:dao)
dpi = 150

# Create a color gradient based on initial ω values
initial_ω = [ic[2] for ic in initial_conditions]
colors = cgrad(:viridis)  # Choose any colormap you like (:viridis, :plasma, :inferno, etc.)

# Normalize initial ω values to [0,1] for colormap
ω_min, ω_max = extrema(initial_ω)
ω_norm = @. (initial_ω - ω_min) / (ω_max - ω_min)

# Assign colors based on initial ω
# point_colors = [colors[ω_norm[i]] for i in 1:n_pendula]

# Alternative: Binary coloring (positive/negative initial ω)
point_colors = [ω > 0 ? :blue : :red for ω in initial_ω]

anim = @animate for i in 1:total_frames
    t = (i - 1) / (total_frames - 1) * tspan[2]

    # Extract current state of all pendula at this frame
    θs = [sol[1, i] for sol in solutions]
    ωs = [sol[2, i] for sol in solutions]

    # Wrap θ to [-π, π] for plotting
    θs_wrapped = mod.(θs .+ π, 2π) .- π

    # Create phase space plot
    p = scatter(θs_wrapped, ωs,
        xlims=(-π, π), ylims=(-2.5, 2.5),
        xlabel=L"\theta", ylabel=L"\dot{\theta}",
        title="Pendula Phase Space Mixing (t = $(@sprintf(" % 0.2f", t)))",
        legend=false,
        markersize=8,
        markercolor=point_colors,  # Use our color assignments
        size=(800, 600),
        titlefontsize=20,
        tickfontsize=12,
        legendfontsize=10,
        yguidefontsize=15,
        xguidefontsize=15,
        dpi=dpi,
        alpha=0.5)

    # Add multiple energy contours
    θ_range = range(-π, π, length=100)
    for E in [0.25, 0.5, 1.0, 1.5, 2.0, 2.5]
        ω_pos = @. sqrt(max(0, 2 * (E + cos(θ_range))))
        ω_neg = @. -sqrt(max(0, 2 * (E + cos(θ_range))))
        valid = 2 * (E .+ cos.(θ_range)) .≥ 0
        plot!(θ_range[valid], ω_pos[valid],
            color=:black, linestyle=:dash, linewidth=1.5, alpha=0.7, label="")
        plot!(θ_range[valid], ω_neg[valid],
            color=:black, linestyle=:dash, linewidth=1.5, alpha=0.7, label="", dpi=dpi)
    end

    # Add quiver plot
    #=
    θ_grid = range(-π, π, length=15)
    ω_grid = range(-2.5, 2.5, length=15)
    θ_quiver, ω_quiver = meshgrid(θ_grid, ω_grid)
    dθ = ω_quiver
    dω = -sin.(θ_quiver)
    norm_factor = @. sqrt(dθ^2 + dω^2)
    dθ_norm = dθ ./ (norm_factor .+ 1e-6) * 0.2
    dω_norm = dω ./ (norm_factor .+ 1e-6) * 0.2
    quiver!(θ_quiver[:], ω_quiver[:], quiver=(dθ_norm[:], dω_norm[:]),
        color=:black, alpha=1.0, linewidth=1.5, arrow=arrow(:closed, :head, 0.05, 0.2))
    =#

    # Highlight the main separatrix
    main_sep_ω = 2 .* cos.(θ_range ./ 2)
    plot!(θ_range, main_sep_ω, color=:black, linewidth=1.5, label="Main Separatrix")
    plot!(θ_range, -main_sep_ω, color=:black, linewidth=1.5, label="", dpi=dpi)

    p
end
=#

In [9]:
# Prepare the animation and save individual plots
println("Creating animation and saving individual frames...")
Plots.theme(:dao)
dpi = 150

# Create output directory if it doesn't exist
output_dir = "pendulum_frames"
if !isdir(output_dir)
    mkdir(output_dir)
end

# Create a color gradient based on initial ω values
initial_ω = [ic[2] for ic in initial_conditions]
colors = cgrad(:viridis)  # Choose any colormap you like (:viridis, :plasma, :inferno, etc.)

# Normalize initial ω values to [0,1] for colormap
ω_min, ω_max = extrema(initial_ω)
ω_norm = @. (initial_ω - ω_min) / (ω_max - ω_min)

# Assign colors based on initial ω
# point_colors = [colors[ω_norm[i]] for i in 1:n_pendula]

# Alternative: Binary coloring (positive/negative initial ω)
point_colors = [ω > 0 ? :blue : :red for ω in initial_ω]

anim = @animate for i in 1:total_frames
    t = (i - 1) / (total_frames - 1) * tspan[2]

    # Extract current state of all pendula at this frame
    θs = [sol[1, i] for sol in solutions]
    ωs = [sol[2, i] for sol in solutions]

    # Wrap θ to [-π, π] for plotting
    θs_wrapped = mod.(θs .+ π, 2π) .- π

    # Create phase space plot
    p = scatter(θs_wrapped, ωs,
        xlims=(-π, π), ylims=(-2.5, 2.5),
        xlabel=L"\theta", ylabel=L"\dot{\theta}",
        title="Pendula Phase Space Mixing (t = $(@sprintf(" % 0.2f", t)))",
        legend=false,
        markersize=8,
        markercolor=point_colors,  # Use our color assignments
        size=(800, 600),
        titlefontsize=20,
        tickfontsize=12,
        legendfontsize=10,
        yguidefontsize=15,
        xguidefontsize=15,
        dpi=dpi,
        alpha=0.5)

    # Add multiple energy contours
    θ_range = range(-π, π, length=100)
    for E in [0.25, 0.5, 1.0, 1.5, 2.0, 2.5]
        ω_pos = @. sqrt(max(0, 2 * (E + cos(θ_range))))
        ω_neg = @. -sqrt(max(0, 2 * (E + cos(θ_range))))
        valid = 2 * (E .+ cos.(θ_range)) .≥ 0
        plot!(θ_range[valid], ω_pos[valid],
            color=:black, linestyle=:dash, linewidth=1.5, alpha=0.7, label="")
        plot!(θ_range[valid], ω_neg[valid],
            color=:black, linestyle=:dash, linewidth=1.5, alpha=0.7, label="", dpi=dpi)
    end

    # Highlight the main separatrix
    main_sep_ω = 2 .* cos.(θ_range ./ 2)
    plot!(θ_range, main_sep_ω, color=:black, linewidth=1.5, label="Main Separatrix")
    plot!(θ_range, -main_sep_ω, color=:black, linewidth=1.5, label="", dpi=dpi)

    # Save individual frame
    frame_filename = joinpath(output_dir, "frame_$(lpad(i, 4, '0')).png")
    savefig(p, frame_filename)
    
    p
end

# Create and save the animation
gif(anim, "pendulum_animation.gif", fps=30)
println("Animation saved as pendulum_animation.gif")
println("Individual frames saved in $output_dir directory")

Creating animation and saving individual frames...


[ Info: Saved animation to /home/belster/JuliaFiles/Misc/pendulum_animation.gif


Animation saved as pendulum_animation.gif
Individual frames saved in pendulum_frames directory


In [10]:
#=
# Prepare the animation
println("Creating animation...")
Plots.theme(:dao)

anim = @animate for i in 1:total_frames
    t = (i - 1) / (total_frames - 1) * tspan[2]

    # Extract current state of all pendula at this frame
    θs = [sol[1, i] for sol in solutions]
    ωs = [sol[2, i] for sol in solutions]

    # Wrap θ to [-π, π] for plotting
    θs_wrapped = mod.(θs .+ π, 2π) .- π

    # Create phase space plot
    scatter(θs_wrapped, ωs,
        xlims=(-π, π), ylims=(-2.5, 2.5),
        xlabel="θ", ylabel=L"\dot{\theta}",
        title="Pendula Phase Space Mixing (t = $(@sprintf(" % 0.2f", t)))",
        legend=false, markersize=4,
        size=(800, 600),
        titlefontsize=20,
        tickfontsize=12,
        legendfontsize=10,
        yguidefontsize=15,
        xguidefontsize=15,)

    # Add separatrix (the boundary between libration and rotation)
    separatrix_θ = range(-π, π, length=100)
    separatrix_ω = 2 .* cos.(separatrix_θ ./ 2)
    plot!(separatrix_θ, separatrix_ω, color=:black, linestyle=:dash, linewidth=3, label="Separatrix")
    plot!(separatrix_θ, -separatrix_ω, color=:black, linestyle=:dash, linewidth=3, label="")
end
=#

In [11]:
# Save the animation
mp4(anim, "pendulum_phase_mixing.mp4", fps=fps)
println("Animation saved to pendulum_phase_mixing.mp4")

Animation saved to pendulum_phase_mixing.mp4


[ Info: Saved animation to /home/belster/JuliaFiles/Misc/pendulum_phase_mixing.mp4
